In [283]:
import time

import mysql.connector
from Params import hostSQL, userSQL, passwdSQL, databaseSQL
import os
import json
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt 

import nltk
from nltk.corpus import stopwords

# --------------------------------------------------
 
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.cluster import KMeans

# --------------------------------------------------
import warnings
warnings.filterwarnings("ignore",category=DeprecationWarning)



In [284]:

mydb = mysql.connector.connect(
    host=hostSQL,
    user=userSQL,
    passwd=passwdSQL,
    auth_plugin='mysql_native_password',
    database=databaseSQL
)

mycursor = mydb.cursor()


In [285]:
sql = 'SELECT * FROM chatroom'
mycursor.execute(sql)
chatlog = mycursor.fetchall()

sql = 'DESCRIBE chatroom'
mycursor.execute(sql)
chatDescribe = mycursor.fetchall()

columns = []
for i in chatDescribe:
    columns.append(i[0])

df = pd.DataFrame(chatlog, columns=columns)

df['chat_line'] = df.chat_line.str[:-1]



In [286]:

stop_words = stopwords.words('english')

stop_words.extend([
    'liv',
    'im',
    'nicole',
    'heyits127joepuckers',
    'hottea1dab',
    'mercywing2',
    'mercywing1',
    'trae',
    'leylaphype',
    'leylapmario',
    'mariioo007',
    'leylaplove',
    'leylakatana',
    'footpics23',
    'rpgghosto',
    'kill199023',
    'slawscare',
    'eclairehyperbruh',
    'traebear',
    'ajvielul',
    'bryce',
    'pansexualpride',
    'hottea1tier1',
    'hottea1hottea',
    'jaina',
    'andyraemodlove',
    'lylythfaire',
    'ajvieflex',
    'leylapcozy',
    'leylaplurk',
    'evelyn',
    'cass',
    'mrandyp',
    'fizzle27',
    'laribump',
    'fbcatch',
    'evelique',
    'bankshot1321',
    'meg',
    'ellen',
    'james',
    'yogii',
    'lari',
    'livstixs',
    'bandslam',
    'hot_tea101',
    'araynebow',
    'nice'
])

stop_words.extend(dfChat.display_name.unique().tolist())

channels = [
            # 'ajvie',
            # 'ayytrae',
            # 'child_of_christ666',
            # 'davinityyy',
            # 'emandliv',
            # 'hollytwolf',
            # 'hottea',
            # 'kneecoleslaw',
            # 'laribug',
            # 'leyla',
            # 'livstixs',
            # 'lizelda',
            'martinimonsters'
            # 'miss_angeliquew',
            # 'misdelish',
            # 'nataagataa',
            # 'officialevelynclaire',
            # 'pink_sparkles',
            # 'xoaeriel',
            # 'yogiibutt'
            ]


dfChat = df[df.chatchannel.isin(channels)]

dfChat = dfChat[dfChat['display_name'] != 'nightbot']

renameDict = {'chat_line': 'text'}
dfChat.rename(columns=renameDict, inplace=True)

data = dfChat.text.tolist()

In [287]:
from nltk.tokenize import sent_tokenize, word_tokenize
from nltk.stem import PorterStemmer

porter=PorterStemmer()

def stemSentence(sentence):
    token_words=word_tokenize(sentence)
    token_words
    stem_sentence=[]
    for word in token_words:
        stem_sentence.append(porter.stem(word))
        stem_sentence.append(" ")
    return "".join(stem_sentence)

for i in range(len(data)):
    data[i] = stemSentence(data[i])

vectorizer = TfidfVectorizer(stop_words=stop_words)
X = vectorizer.fit_transform(data)


np.random.seed(200)

clusters = 30
model = KMeans(n_clusters=clusters, init='k-means++', max_iter=100, n_init=1)
model.fit(X)

order_centroids = model.cluster_centers_.argsort()[:, ::-1]
terms = vectorizer.get_feature_names()

dfChat = dfChat.reset_index()

labels = pd.DataFrame(model.labels_, columns=['label'])

dfChatwLabels = pd.concat([dfChat.iloc[:,1:], labels], axis=1)

dfChatwLabels.groupby(['label']).count()

,from_ID,display_name,text,sent_at,chatchannel
label,,,,,
0,2,2,2,2,2
1,270,270,270,270,270
2,184,184,184,184,184
3,14,14,14,14,14
4,64,64,64,64,64
5,20370,20370,20370,20370,20370
6,56,56,56,56,56
7,42,42,42,42,42
8,139,139,139,139,139


In [288]:
for i in range(clusters):
    print('Cluster %d:' % i),
    for ind in order_centroids[i, :10]:
        print(' %s' % terms[ind])

Cluster 0:
 prepar
 question
 test
 dumb
 zzzz
 fyi
 gabibbo
 gabacho
 ga
 fzn666
Cluster 1:
 lol
 wtf
 whi
 wa
 good
 hilari
 get
 orang
 like
 happi
Cluster 2:
 hello
 beauti
 itali
 everyon
 ladi
 liz
 miss
 rank
 love
 germani
Cluster 3:
 normal
 tho
 ahah
 snot
 expect
 plant
 christma
 trachea
 thumb
 thing
Cluster 4:
 danc
 pole
 sub
 start
 like
 lap
 love
 nicol
 pleas
 club
Cluster 5:
 love
 hi
 like
 thi
 good
 get
 clap
 wa
 go
 look
Cluster 6:
 help
 pleas
 fix
 consequ
 record
 recent
 nude
 gross
 jerk
 time
Cluster 7:
 milk
 stepmommi
 want
 pour
 titti
 fresh
 step
 mommi
 drink
 need
Cluster 8:
 omg
 queen
 hi
 jame
 ty
 kill
 real
 lol
 clap
 beauti
Cluster 9:
 ok
 oh
 sorri
 wa
 cool
 lol
 hope
 simp
 bud
 need
Cluster 10:
 came
 back
 stay
 cake
 gold
 smart
 drop
 game
 found
 time
Cluster 11:
 sr
 http
 halsey
 bad
 hhjhykpq8f0
 love
 front
 request
 put
 fwend
Cluster 12:
 http
 instagram
 follow
 go
 insta
 futureman
 futurist
 gabslov
 gabscut
 gabibbo
Cluster

In [289]:
testMsg = ['ur a simp']

testMsg = [stemSentence(testMsg[0])]

print('Prediction')
X = vectorizer.transform(testMsg)
predicted = model.predict(X)
print(predicted)

dfChatwLabels[dfChatwLabels['label'] == predicted[0]].text.tolist()

Prediction
[5]


gHey pugHey pugHey pugWMA',
 'sumMhm sumMhm sumMhm',
 'OMEGALUL',
 'Sage might be amazing for you, Nicole.',
 'whats the go about the stimulus check graphic?',
 'kaypikeHeyGuys',
 'classic girl stream... gameplay secondary mini screen, b33bz main focus OMEGALUL',
 'WE NEED THE KEY',
 'Can you make the gameplay a bit smaller',
 'that view was better!',
 'too many thirsty fuckers in here',
 'what?',
 'hows that thirsty',
 'it was better gameplay ion give a shit abt her tits homie',
 'not you quezy haha',
 'Free those b00bs',
 'oh sorry homie',
 'all G',
 'its fuckin blazin in my room hotboxed and its makin me angry LOL',
 'Is there friendly fire in this game',
 'haha. my tourettes makes me angry enough haha',
 'ahhh',
 'kfcclaLove kfcclaLove',
 'checkin in on the twins - lookin good still - c u next month',
 'where is my beta key',
 'coupled with night shift and lack of sleeps',
 'SeemsGood',
 'They are',
 'where is it',
 'gotta wait',
 'fake B00bs op',
 'gotta watch stream for like 2 ho

In [290]:
clusterNum = 3

dfChatwLabels[dfChatwLabels['label'] == clusterNum].text.tolist()

['normal',
 'now its normal tho',
 'things wont be back t normal for yeas get used to t',
 'even tho the cake didnt come out normal ive been entertained emzGG',
 'you have high expectations so its normal',
 "I think your thumbs are normal. It's your trachea that is fucked",
 'u sure u are normal? ahah',
 '@JRock_TTV ur a normal champ congrats being normal so cool',
 'after you create your onlyfans account everything will return to normal as god intended.',
 "You have a normal forehead don't worry and beautiful lips",
 'Brenden watched you in the shower and doesnt get blood rushing to his pp ... is he normal?',
 'Wait that snot normal haha',
 "it's not christmas, just have a normal plant",
 'and its normal, i spent years....years wasting my life dwelling on the shitty past with my ex, yet now, if i see her, i can talk to her like shes just a normal person, not saying you will or ever have to do that ha, but it shows how things can change just like that...so...anyway ill shut up now']

In [291]:
martonoChat = df[df['chatchannel'] == 'martinimonsters']

from wordcloud import WordCloud, STOPWORDS

comment_words = '' 
stopwords = set(STOPWORDS) 
  
# iterate through the csv file 
for val in martonoChat.text: 
      
    # typecaste each val to string 
    val = str(val) 
  
    # split the value 
    tokens = val.split() 
      
    # Converts each token into lowercase 
    for i in range(len(tokens)): 
        tokens[i] = tokens[i].lower() 
      
    comment_words += " ".join(tokens)+" "
  
wordcloud = WordCloud(width = 2000, height = 2000, 
                background_color ='white', 
                stopwords = stopwords, 
                min_font_size = 10).generate(comment_words) 
  
# plot the WordCloud image                        
plt.figure(figsize = (15, 15), facecolor = None) 
plt.imshow(wordcloud) 
plt.axis("off") 
plt.tight_layout(pad = 0) 
  
plt.show() 

AttributeError: 'DataFrame' object has no attribute 'text'